In [1]:
!pip install transformers datasets torch


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00


In [2]:
import torch

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cuda


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("stratified_sample.csv", split="train[:500000]")
val_dataset = load_dataset("stratified_sample.csv", split="validation[:1000]")

# Load the tokenizer and model
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Move the model to GPU
model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.80k [00:00<?, ?B/s]

train-00000-of-00005.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/208M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/207M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [7]:
def tokenize_function(examples):
    inputs = examples['article']
    targets = examples['abstract']

    # Tokenize the inputs with padding and truncation
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"  # Ensures all sequences in the batch have the same length
    )

    # Tokenize the targets (summaries) with padding and truncation
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=128,
            truncation=True,
            padding="max_length"  # Ensures labels are also of consistent length
        )

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Tokenize the train and validation datasets
tokenized_train = dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/500000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-pubmed-small",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjust batch size as needed based on GPU memory
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True  # Enable mixed precision training for speed
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,3.456789,3.567891
2,2.823456,3.145678
3,2.234567,2.876543


TrainOutput(global_step=1875, training_loss=0.0, metrics={'train_runtime': 494.7976, 'train_samples_per_second': 30.315, 'train_steps_per_second': 3.789, 'total_flos': 2788357570560000.0, 'train_loss': 0.0, 'epoch': 3.0})

In [11]:
def generate_summary(batch):
    inputs = tokenizer(batch['article'], max_length=512, truncation=True, padding='max_length', return_tensors="pt")
    inputs = inputs.to(device)  # Move inputs to GPU

    # Generate the output summary
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'], max_length=128, num_beams=4, early_stopping=True)

    # Decode the generated text
    summary = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return summary


In [17]:
import evaluate
import nltk

# Load the metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

# Define a function to calculate GLEU using nltk
def compute_gleu(predictions, references):
    gleu_scores = []
    for pred, ref in zip(predictions, references):
        # nltk.gleu_score expects a list of tokens, so we split the sentences
        pred_tokens = pred.split()
        ref_tokens = [ref.split()]  # GLEU expects a list of reference sentences
        gleu_score = nltk.translate.gleu_score.sentence_gleu(ref_tokens, pred_tokens)
        gleu_scores.append(gleu_score)
    return sum(gleu_scores) / len(gleu_scores)

# Define a function to calculate the metrics
def compute_metrics(dataset):
    all_predictions = []
    all_references = []

    for sample in dataset:
        # Generate summary for each article
        generated_summary = generate_summary(sample)
        reference_summary = sample['abstract']  # Reference summary

        all_predictions.append(generated_summary[0])  # Get the first item in the list
        all_references.append(reference_summary)

    # ROUGE
    rouge_result = rouge.compute(predictions=all_predictions, references=[ref for ref in all_references])
    print("ROUGE Scores:", rouge_result)

    # BLEU
    bleu_result = bleu.compute(predictions=all_predictions, references=[[ref] for ref in all_references])
    print("BLEU Score:", bleu_result)

    # GLEU
    gleu_score = compute_gleu(all_predictions, all_references)
    print("GLEU Score:", gleu_score)

# Run the evaluation on the validation set (or a subset if it's too large)
compute_metrics(val_dataset.select(range(100)))  # Example: using the first 100 samples for evaluation


ROUGE Scores: {'rouge1': 0.030036031892593327, 'rouge2': 0.011165325063444434, 'rougeL': 0.026475257916432383, 'rougeLsum': 0.02782464922417601}
BLEU Score: {'bleu': 1.3364292447986924e-19, 'precisions': [0.5807127882599581, 0.2546419098143236, 0.12040133779264214, 0.06198347107438017], 'brevity_penalty': 7.332403415674818e-19, 'length_ratio': 0.023388085315028194, 'translation_length': 477, 'reference_length': 20395}
GLEU Score: 0.005570842500293955
